In [13]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
ruta_base = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ruta_base not in sys.path:
    sys.path.append(ruta_base)
from Modulo import *


In [ ]:
# Constantes
g = 9.81         # Aceleración gravitacional (m/s^2)
k = 8.99e9       # Constante de Coulomb (N·m²/C²)

# ======================
# Datos experimentales
# ======================

# --- Datos para la bola de poliuretano ---
mass_poly = np.array([
    3e-3, 3e-3, 3.1e-3, 3e-3, 3.1e-3
])
angle_poly = np.array([
    # TODO: Ingresa las mediciones de ángulo en grados
])
distance_poly = np.array([
    # TODO: Ingresa las mediciones de distancia en m
])

# --- Datos para la bola metálica ---
mass_met = np.array([
    2.7e-3, 2.6e-3, 2.6e-3, 2.7e-3, 2.6e-3 #Kg
])
angle_met = np.array([
     #Grados
])
distance_met = np.array([
     #m
])

# Si el error asociado es el mismo para todas las mediciones, definimos un valor y generamos arrays de error:
error_mass_value = 0.05e-3      # TODO: Ingresa el error en masa (kg)
error_angle_value = 0.0     # TODO: Ingresa el error en ángulo (grados)
error_distance_value = 0.0  # TODO: Ingresa el error en distancia (m)

error_mass_poly = np.full(len(mass_poly), error_mass_value)
error_angle_poly = np.full(len(angle_poly), error_angle_value)
error_distance_poly = np.full(len(distance_poly), error_distance_value)

error_mass_met = np.full(len(mass_met), error_mass_value)
error_angle_met = np.full(len(angle_met), error_angle_value)
error_distance_met = np.full(len(distance_met), error_distance_value)

# ======================================
# Cálculo de promedios y sus errores
# ======================================

mass_poly_avg, mass_poly_err = error_mean(mass_poly, error_mass_poly)
angle_poly_avg, angle_poly_err = error_mean(angle_poly, error_angle_poly)
distance_poly_avg, distance_poly_err = error_mean(distance_poly, error_distance_poly)

mass_met_avg, mass_met_err = error_mean(mass_met, error_mass_met)
angle_met_avg, angle_met_err = error_mean(angle_met, error_angle_met)
distance_met_avg, distance_met_err = error_mean(distance_met, error_distance_met)

# ======================================
# Cálculo de la fuerza electrostática
# Usando la relación: F = m * g * tan(theta)
# (Asegurarse de convertir el ángulo a radianes)
# ======================================

# Convertir ángulos promedio y sus errores a radianes
angle_poly_rad = np.deg2rad(angle_poly_avg)
angle_met_rad = np.deg2rad(angle_met_avg)
angle_poly_err_rad = np.deg2rad(angle_poly_err)
angle_met_err_rad = np.deg2rad(angle_met_err)

def calc_force(mass, mass_err, angle_rad, angle_err_rad):
    """
    Calcula la fuerza F = m * g * tan(theta) y propaga los errores.
    """
    force = mass * g * np.tan(angle_rad)
    # Derivadas parciales:
    # dF/dm = g * tan(theta)
    # dF/dθ = m * g * sec^2(theta)
    sec_theta = 1 / np.cos(angle_rad)
    dF_dm = g * np.tan(angle_rad)
    dF_dtheta = mass * g * sec_theta**2
    force_err = np.sqrt((dF_dm * mass_err)**2 + (dF_dtheta * angle_err_rad)**2)
    return force, force_err

force_poly, force_poly_err = calc_force(mass_poly_avg, mass_poly_err, angle_poly_rad, angle_poly_err_rad)
force_met, force_met_err = calc_force(mass_met_avg, mass_met_err, angle_met_rad, angle_met_err_rad)

# ======================================
# Cálculo de la carga asociada (q)
# Usando la ley de Coulomb: F = k * q^2 / r^2   =>   q = sqrt(F * r^2 / k)
# ======================================

def calc_charge(force, force_err, distance, distance_err):
    """
    Calcula la carga q y propaga el error, usando:
      q = sqrt( F * r^2 / k )
    """
    charge = np.sqrt(force * distance**2 / k)
    if charge == 0:
        charge_err = 0
    else:
        # Derivadas parciales:
        # dq/dF = (1/(2*q)) * (r^2/k)
        # dq/dr = (1/(2*q)) * (2*F*r/k) = (F*r)/(q*k)
        dq_dF = (distance**2 / k) / (2 * charge)
        dq_dr = (force * distance / k) / charge
        charge_err = np.sqrt((dq_dF * force_err)**2 + (dq_dr * distance_err)**2)
    return charge, charge_err

charge_poly, charge_poly_err = calc_charge(force_poly, force_poly_err, distance_poly_avg, distance_poly_err)
charge_met, charge_met_err = calc_charge(force_met, force_met_err, distance_met_avg, distance_met_err)

# ======================================
# Mostrar resultados en consola
# ======================================

print("----- Resultados para la bola de poliuretano -----")
print(f"Masa promedio: {mass_poly_avg:.4f} ± {mass_poly_err:.4f} kg")
print(f"Ángulo promedio: {angle_poly_avg:.2f} ± {angle_poly_err:.2f} grados")
print(f"Distancia promedio: {distance_poly_avg:.4f} ± {distance_poly_err:.4f} m")
print(f"Fuerza calculada: {force_poly:.4f} ± {force_poly_err:.4f} N")
print(f"Carga calculada: {charge_poly:.4e} ± {charge_poly_err:.4e} C")

print("\n----- Resultados para la bola metálica -----")
print(f"Masa promedio: {mass_met_avg:.4f} ± {mass_met_err:.4f} kg")
print(f"Ángulo promedio: {angle_met_avg:.2f} ± {angle_met_err:.2f} grados")
print(f"Distancia promedio: {distance_met_avg:.4f} ± {distance_met_err:.4f} m")
print(f"Fuerza calculada: {force_met:.4f} ± {force_met_err:.4f} N")
print(f"Carga calculada: {charge_met:.4e} ± {charge_met_err:.4e} C")

# ======================================
# Gráfica
# ======================================
inv_distance2_poly = 1 / distance_poly**2

plt.figure(figsize=(8, 6))
plt.errorbar(inv_distance2_poly, mass_poly, fmt='o', capsize=5, label='Datos poliuretano')
plt.xlabel('1/(Distancia²) [1/m²]')
plt.ylabel('Masa (kg)')
plt.title('Relación entre masa y 1/(distancia²) para la bola de poliuretano')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
plt.errorbar(inv_distance2_poly, mass_poly, fmt='o', capsize=5, label='Datos poliuretano')
plt.xlabel('1/(Distancia²) [1/m²]')
plt.ylabel('Masa (kg)')
plt.title('Relación entre masa y 1/(distancia²) para la bola de poliuretano')
plt.legend()
plt.grid(True)

graficas_dir = os.path.join(os.getcwd(), "Graficas")
if not os.path.exists(graficas_dir):
    os.makedirs(graficas_dir)
    
grafica_path = os.path.join(graficas_dir, "grafica_reporte_1.png")
plt.savefig(grafica_path, dpi=300)
print(f"Gráfica guardada en: {grafica_path}")

plt.show()


Suma: 10.50 ± 0.23
